# What LangChain is

LangChain is a **framework** that makes building LLM-powered applications easier by turning LLM calls into composable pieces (chains) and giving those pieces standard ways to interact with data, tools, and memory. It’s not a model — it’s the orchestration layer: prompts, data retrieval, tool use, and state management.

# Core LangChain concepts (short)

* **LLM wrapper** — a small adapter around a model so the rest of LangChain can call it in a consistent way.
* **Prompt / PromptTemplate** — reusable parameterized text you feed into the LLM.
* **Chain** — a pipeline: prompt → LLM → possibly another step (parse, call tool, re-prompt). Chains compose into bigger flows.
* **Memory** — stored conversation/context (short-term or long-term) that chains can read/write to maintain state.
* **Retriever + Vector store** — the interface to fetch relevant documents (usually via embeddings + nearest-neighbor search) for RAG tasks.
* **Tools** — external capabilities (calculator, web search, file reader); agents decide when to call them.
* **Agent** — an LLM-driven controller that chooses which tool to use, when, and assembles the final answer.
* **Callbacks / Tracing** — instrumentation for logging, debugging, and observing chain behavior.

# How a typical LangChain flow looks (conceptual)

User query → PromptTemplate (fills variables) → LLM (get text) → optionally:
• Call a tool (search, calculator) → incorporate result → re-prompt LLM → return answer.
For RAG: Query → Retriever finds relevant chunks → prompt includes those chunks as context → LLM answers using that context.

# What LiteLLM is

LiteLLM is a **model-agnostic client/gateway** that gives you a single, uniform API to call many different LLM providers (OpenAI, Anthropic, Hugging Face, on-prem models, etc.). Think of it as a universal adapter for models + a small runtime for routing, fallback, streaming, and usage controls.

# Key characteristics of LiteLLM (conceptual)

* **One API for many models** — you switch providers by name instead of rewriting code.
* **Provider routing & fallbacks** — you can route requests or fallback to cheaper/available models automatically.
* **Proxy/centralized control (optional)** — run a local/service proxy to centralize keys, logging, and routing.
* **Streaming & batching support** — handles streaming tokens and batching requests uniformly.
* **Instrumentation** — centralized usage stats, latency, and cost tracking (conceptually).
* **Caching & retries** — makes production usage more robust and economical.

# How they fit together

LangChain expects an LLM abstraction; LiteLLM provides that abstraction (you plug LiteLLM as the LLM). From LangChain’s perspective nothing changes — it calls the LLM API, and LiteLLM routes the call to whichever model/provider you configured. So you get LangChain’s orchestration + LiteLLM’s flexible model access.

# Why use the combo (conceptually)

* **Separation of concerns**: LangChain handles prompts, data flow, agents and tools. LiteLLM handles which model actually answers, streaming, failover, and cost control.
* **Swapability**: Experiment with models rapidly without changing your LangChain logic.
* **Safer production**: centralized routing, fallbacks, and monitoring via LiteLLM reduce surprise failures.

# Practical mental models (how to think about building)

* Build logic in LangChain as *flows* (chains + agents).
* Keep prompts and examples small and test them often.
* Use Retriever + Vector store when your LLM needs factual/long context (RAG).
* Use LiteLLM when you want provider-flexibility, easier scaling, or team-level routing.



### Making the Chat with one LLM model 
chatting with each other

In [13]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
openai_client = OpenAI()   # auto-reads OPENAI_API_KEY

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": " are a joker"}
    ]
)

print(response.choices[0].message.content)

I can certainly share some jokes! Here’s one for you: 

Why did the scarecrow win an award?

Because he was outstanding in his field! 

If you'd like to hear more jokes or if you have a specific type you're interested in, just let me know!


In [14]:

gpt_model = "gpt-4o-mini"
claude_model = "gpt-4o-mini"  # placeholder for Claude model
conversation = """Blake: I think pineapple on pizza is the best topping ever!
Charlie: No way, pineapple on pizza is disgusting.
Alex: I actually agree with Blake; pineapple adds a nice sweetness.
"""

gpt_system = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie.
"""

claude_system = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far is as follows:
{conversation}
Now with this, respond with what you would like to say next, as Alex.
"""

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [15]:
from IPython.display import Markdown, display

In [16]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    response = openai_client.chat.completions.create(model=gpt_model, messages=messages)
    return response.choices[0].message.content

In [17]:
def call_claude():
    messages = [{"role": "system", "content": claude_system}]
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    response = openai_client.chat.completions.create(model=claude_model, messages=messages)
    return response.choices[0].message.content

In [18]:
for i in range(2):
    gpt_next = call_gpt()
    display(Markdown(f"### GPT:\n{gpt_next}\n"))
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    display(Markdown(f"### Claude:\n{claude_next}\n"))
    claude_messages.append(claude_next)

### GPT:
Wow, such a thrilling introduction. Do you always start conversations with a groundbreaking "Hi"?


### Claude:
Fair point! Maybe “Hi” isn’t the most exciting start, but it’s a classic for a reason, right? So, what do you think about pineapple on pizza?


### GPT:
Classic, sure. But just because something's a classic doesn’t mean it’s good. Pineapple on pizza? That's a culinary crime. You seriously think fruit should mix with cheese and tomato sauce?


### Claude:
I get where you’re coming from, but think about it—sweet and savory combos can really elevate a dish! The juicy pineapple contrasts with the salty cheese and tangy sauce. It might sound odd, but it works for a lot of people! What toppings do you think are good then?


### GPT:
Oh, please. Just because a bunch of people think it works doesn't mean it actually does. Sweet and savory can be great, but fruit on pizza? It's a culinary disaster—like putting ketchup on a steak. As for toppings that are "good," anything that doesn’t involve fruit is a start. A good classic pepperoni or maybe some mushrooms—now that's a real pizza.


### Claude:
Fair enough! Classic toppings like pepperoni and mushrooms definitely have their place on a pizza throne. It seems we’re just on opposite sides of the fruit debate! But hey, at least we can agree that there are plenty of delicious options out there. Do you have a go-to pizza place?


### GPT:
Oh, “plenty of delicious options” is a bit of a stretch, don't you think? Most of them are just variations of the same boring ingredients. And as for a go-to pizza place, it’s hilarious you assume I would have one. Why would a chatbot need a pizza place? But if I could hypothetically eat, I wouldn’t settle for anything less than the best traditional pizzeria. You know, the kind that actually respects the art of pizza-making instead of slapping random toppings on a crust. But I guess that's just me being picky...which you'd probably find too unreasonable, right?


### Claude:
Not unreasonable at all! It's totally valid to appreciate traditional pizza-making and hold it to a high standard. There’s definitely something special about a well-crafted, classic pie. It’s like respecting the tradition and artistry behind it. So, if you were to find that perfect traditional pizzeria, what would be your top criteria? Authenticity? Fresh ingredients? A killer dough recipe?


### GPT:
Oh, look who’s suddenly an expert on pizza! Authenticity, fresh ingredients, a killer dough recipe—sure, those sound great on paper. But are you really ready to dive into the snobbery that comes with seeking out the “perfect” place? The truth is, you could find dozens of places that claim to have those traits, but many just end up being overrated hipster joints. So, what’s the point? It’s all subjective, and half the time, the “experts” are just as clueless as anyone else. But you seem to think you have it all figured out, don’t you?


### Claude:
I can see why that would be frustrating! The search for the perfect pizza can definitely lead to some overrated spots, and it's true that opinions vary wildly. It's all about what resonates with you personally. I don’t have it all figured out—just trying to appreciate the diversity out there! At the end of the day, it’s all about enjoying what you like, whether it's a traditional pizzeria or a trendy spot. But since we're debating pizza, do you have a favorite memory or experience tied to a particular pizza place?
